In [1]:
! pip install transformers -q

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 636 kB 44.3 MB/s 
     |████████████████████████████████| 3.3 MB 37.0 MB/s 
     |████████████████████████████████| 895 kB 44.3 MB/s 


In [2]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()

In [4]:
df = pd.read_csv(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Datasets/LIAR/train.tsv", 
                 sep = "\t",
                 names = ['ID', 'label', 'statement', 'subject', 'speaker','speaker_job',
                          'state', 'party', 'barely true counts', 'false counts',
                          'half true counts', 'mostly true counts', 'pants on fire counts',
                          'context'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10240 entries, 0 to 10239
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    10240 non-null  object 
 1   label                 10240 non-null  object 
 2   statement             10240 non-null  object 
 3   subject               10238 non-null  object 
 4   speaker               10238 non-null  object 
 5   speaker_job           7343 non-null   object 
 6   state                 8032 non-null   object 
 7   party                 10238 non-null  object 
 8   barely true counts    10238 non-null  float64
 9   false counts          10238 non-null  float64
 10  half true counts      10238 non-null  float64
 11  mostly true counts    10238 non-null  float64
 12  pants on fire counts  10238 non-null  float64
 13  context               10138 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


In [6]:
'''
Here we check the domain of the news from business, education, entertainment, political, sport, technology.

seqeunces_to_classify = sentences
cadidate_labels = ['business', 'education', 'entertainment', 'political', 'sport', 'technology']
results = classifier(sequences_to_classify, candidate_labels, multi_label=True)
'''

def get_domain(text):

  candidate_labels = ['business', 'education', 'entertainment', 'political', 'sport', 'technology']

  results = classifier(text, candidate_labels, multi_label=True)

  return results['labels'][0]

In [7]:
# df['domain'] = df['statement'].progress_apply(get_domain)

In [8]:
df.head()

,ID,label,statement,subject,speaker,speaker_job,state,party,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,context
0,2635.json,false,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [9]:
# Installing Libraries

! pip install -U sentence-transformers -q

     |████████████████████████████████| 85 kB 2.6 MB/s 
     |████████████████████████████████| 1.2 MB 36.0 MB/s 


In [10]:
# Importing Libraries


from sentence_transformers import SentenceTransformer ,  util
import torch
import pandas as pd
import numpy as np
import string
import re
from tqdm.auto import tqdm
tqdm.pandas()

import pickle

In [11]:
# Import the sentence transformer model
# model = SentenceTransformer('ghanashyamvtatti/roberta-fake-news', device = 'cuda')
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1', device = 'cuda')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
def generate_embeddings(sentences):
  
  embeddings = model.encode(sentences)
  embeddings=[torch.from_numpy(item) for item in embeddings]
  
  return embeddings

In [13]:
def convert_to_array(tensor):
  return tensor.detach().cpu().numpy()

In [14]:
def convert_label(label):
  if label in ['mostly-true', 'true']:
    return 0
  else:
    return 1

In [15]:
df['label'] = df['label'].apply(convert_label)

In [16]:
statements = df['statement'].tolist()
embeddings = generate_embeddings(statements)
df['embedding'] = embeddings

In [17]:
df['embedding'] = df['embedding'].apply(convert_to_array)

In [18]:
businessdf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/business.pickle")
educationdf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/education.pickle")
entertainmentdf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/entertainment.pickle")
politicaldf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/political.pickle")
sportdf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/sport.pickle")
technologydf = pd.read_pickle(r"/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Word Embeddings/technology.pickle")

In [19]:
businessdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   word       200 non-null    object
 1   embedding  200 non-null    object
 2   label      200 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 4.8+ KB


In [20]:
! pip install annoy -q

     |████████████████████████████████| 646 kB 5.2 MB/s 


In [21]:
from annoy import AnnoyIndex
import random

f = 768

business = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
education = AnnoyIndex(f, 'angular')
entertainment = AnnoyIndex(f, 'angular')
political = AnnoyIndex(f, 'angular')
sport = AnnoyIndex(f, 'angular')
technology = AnnoyIndex(f, 'angular')

business.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/business.ann')
education.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/education.ann')
entertainment.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/entertainment.ann')
political.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/political.ann')
sport.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/sport.ann')
technology.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Annoy Graphs/technology.ann')
# t.load(r'/content/drive/Shareddrives/[FYP] Fake News Detection/Kogul_Language_Modelling/Lexicon Generation/Embedding_Space.ann') # super fast, will just mmap the file

True

In [22]:
# def get_label(embedding):
#   indexes, distances = t.get_nns_by_vector(embedding, 40, include_distances=True, search_k=-1)
#   # indexes = tree.query(embedding, 50)

#   true, fake = 0, 0
#   # for item in indexes:
#   for i in range(len(indexes)):
#     if words['label'][indexes[i]] == 0:
#       # true+=distances[i]
#       true+=1
#     if words['label'][indexes[i]] == 1:
#       # fake+=distances[i]
#       fake+=1
#   # print(fake, true)
#   if fake>true:
#     return 1
#   else:
#     return 0

In [23]:
# embeddings = df['embedding'].tolist()
# domains = df['domain'].tolist()

# predictions = []

# for i in range(len(domains)):

#   true, fake = 0, 0

#   if domains[i] == 'business':
#     indexes, distances = business.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if businessdf['label'][indexes[i]] == 0:
#         # true+=distances[i]
#         true+=1
#       if businessdf['label'][indexes[i]] == 1:
#         # fake+=distances[i]
#         fake+=1

#   elif domains[i] == 'education':
#     indexes, distances = education.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if educationdf['label'][indexes[i]] == 0:
#         # true+=distances[i]
#         true+=1
#       if educationdf['label'][indexes[i]] == 1:
#         # fake+=distances[i]
#         fake+=1

#   elif domains[i] == 'entertainment':
#     indexes, distances = entertainment.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if entertainmentdf['label'][indexes[i]] == 0:
#         # true+=distances[i]
#         true+=1
#       if entertainmentdf['label'][indexes[i]] == 1:
#         # fake+=distances[i]
#         fake+=1

#   elif domains[i] == 'political':
#     indexes, distances = political.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if politicaldf['label'][indexes[i]] == 0:
#         # true+=distances[i]
#         true+=1
#       if politicaldf['label'][indexes[i]] == 1:
#         # fake+=distances[i]
#         fake+=1

#   elif domains[i] == 'sport':
#     indexes, distances = sport.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if sportdf['label'][indexes[i]] == 0:
#         # true+=distances[i]
#         true+=1
#       if sportdf['label'][indexes[i]] == 1:
#         # fake+=distances[i]
#         fake+=1

#   elif domains[i] == 'technology':
#     indexes, distances = technology.get_nns_by_vector(embeddings[i], 40, include_distances=True, search_k=-1)
#     for i in range(len(indexes)):
#       if technologydf['label'][indexes[i]] == 0:
#         true += (1/distances[i])
#         # true+=1
#       if technologydf['label'][indexes[i]] == 1:
#         fake += (1/distances[i])
#         # fake+=1

#   if fake>true:
#     predictions.append(1)
#   else:
#     predictions.append(0)

In [24]:
# get_label(df['embedding'][400])

In [25]:
# df['prediction'] = df['embedding'].apply(get_label)

In [26]:
# from sklearn.metrics import classification_report, confusion_matrix

# print(classification_report(df['label'].tolist(), predictions))

In [33]:
def get_label(embedding, annoygraph):
  indexes, distances = annoygraph.get_nns_by_vector(embedding, 40, include_distances=True, search_k=-1)
  fake, true = 0, 0

  for i in range(len(indexes)):
    if entertainmentdf['label'][indexes[i]] == 0:
      # true+=distances[i]
      true+=1
    if entertainmentdf['label'][indexes[i]] == 1:
      # fake+=distances[i]
      fake+=1

  if fake>true:
    return "Fake"
  else:
    return "True"

In [35]:
def label_to_text(label):
  if label == 1:
    return "Fake"
  else:
    return "True"

In [52]:
num = 100
print("Statement : " + df['statement'][num] + "\n")
print("Domain of the news item : " + get_domain(df['statement'][num]) + "\n")
print("Actual label of the news item : " + label_to_text(df['label'][num]) + "\n")
print("Predicted label according to political lexicon : " + get_label(df['embedding'][num], political))
print("Predicted label according to business lexicon : " + get_label(df['embedding'][num], business))
print("Predicted label according to education lexicon : " + get_label(df['embedding'][num], education))
print("Predicted label according to entertainment lexicon : " + get_label(df['embedding'][num], entertainment))
print("Predicted label according to sport lexicon : " + get_label(df['embedding'][num], sport))
print("Predicted label according to technology lexicon : " + get_label(df['embedding'][num], technology))

Statement : The Florida Department of Agricultures website FreshFromFlorida.com can be used to get a permit to carry a loaded hidden gun without ever leaving your house.

Domain of the news item : technology

Actual label of the news item : Fake

Predicted label according to political lexicon : True
Predicted label according to business lexicon : True
Predicted label according to education lexicon : True
Predicted label according to entertainment lexicon : Fake
Predicted label according to sport lexicon : True
Predicted label according to technology lexicon : True


In [46]:
df.head(10)

,ID,label,statement,subject,speaker,speaker_job,state,party,barely true counts,false counts,half true counts,mostly true counts,pants on fire counts,context,embedding
0,2635.json,1,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer,"[0.07415165, 0.24631837, -0.22498049, 0.076095..."
1,10540.json,1,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,"[0.05078271, 0.05846826, 0.055765472, -0.00281..."
2,324.json,0,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,"[0.16104124, 0.51700455, 0.073424645, 0.245993..."
3,1123.json,1,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,"[0.011539338, 0.18082099, 0.12630084, -0.37024..."
4,9028.json,1,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,"[-0.19220825, 0.23868145, 0.08288821, 0.276036..."
5,12465.json,0,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,"[0.1389486, 0.061372, -0.14447325, 0.053589147..."
6,2342.json,1,Jim Dunnam has not lived in the district he re...,candidates-biography,republican-party-texas,NaN,Texas,republican,3.0,1.0,1.0,3.0,1.0,a press release.,"[-0.017250637, 0.28029713, 0.24255398, 0.32662..."
7,153.json,1,I'm the only person on this stage who has work...,ethics,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,"a Democratic debate in Philadelphia, Pa.","[-0.12953621, -0.049284663, -0.014926965, -0.4..."
8,5602.json,1,"However, it took $19.5 million in Oregon Lotte...",jobs,oregon-lottery,NaN,NaN,organization,0.0,0.0,1.0,0.0,1.0,a website,"[0.09699851, 0.16410835, -0.0115536265, -0.152..."
9,9741.json,0,Says GOP primary opponents Glenn Grothman and ...,"energy,message-machine-2014,voting-record",duey-stroebel,State representative,Wisconsin,republican,0.0,0.0,0.0,1.0,0.0,an online video,"[0.0974286, 0.5207973, -0.19904216, -0.0303949..."


In [34]:
get_label(df['embedding'][10], political)

'True'